# Baseline_model.ipynb

In [1]:
# load cleaned dataset

In [1]:
import pandas as pd
import numpy as np
pd.set_option('precision', 5)

In [2]:
master_app = pd.read_csv('merged_master_app.csv', low_memory=False)

In [4]:
# forgot to output excluding the auto-generated index column.... so now I'm dropping it. Can be optimized later
master_app = master_app.drop(master_app.columns[0], axis=1)
master_app.describe()

,idnproceeding,idnProceedingAppln,idncase_x,numAppsPerProc,idncase_y,tracid
count,6.14204e+05,6.14204e+05,6.14204e+05,614204.00000,6.14204e+05,598132.00000
mean,2.62991e+06,1.47177e+06,4.17382e+06,2.75667,4.17382e+06,146.29007
std,1.48994e+06,1.48298e+06,1.25460e+06,0.78234,1.25460e+06,85.11874
min,7.50000e+01,9.00000e+00,2.04693e+06,1.00000,2.04693e+06,1.00000
25%,1.53156e+06,3.39074e+05,3.20022e+06,2.00000,3.20022e+06,91.00000
50%,2.55462e+06,6.65568e+05,4.01779e+06,3.00000,4.01779e+06,144.00000
75%,3.74955e+06,3.08623e+06,5.14681e+06,3.00000,5.14683e+06,199.00000
max,6.18304e+06,4.67936e+06,7.20315e+06,14.00000,7.20315e+06,403.00000


In [10]:
# selecting features and target variable. Do we want Appl_Code? or that's just for cleaning/selecting the right types
df = master_app[['Appl_Code', 'Appl_Recd_Date', 'numAppsPerProc', 'osc_date', 'tracid', 'nat', 'dec']]
df.head(10)

,Appl_Code,Appl_Recd_Date,numAppsPerProc,osc_date,tracid,nat,dec
0,ASYL,1994-12-20 00:00:00,3.0,1994-11-03,NaN,HO,DENY
1,ASYL,1995-04-02 00:00:00,3.0,1994-11-04,71.0,HO,DENY
2,ASYL,1995-05-05 00:00:00,2.0,1994-11-05,139.0,GT,GRANT
3,ASYL,1995-03-31 00:00:00,3.0,1994-11-05,70.0,ES,DENY
4,ASYL,1995-03-06 00:00:00,3.0,1995-02-06,50.0,HO,DENY
5,ASYL,1996-05-14 00:00:00,2.0,1995-02-06,126.0,CU,GRANT
6,ASYL,1995-11-29 00:00:00,3.0,1995-02-10,71.0,NU,DENY
7,ASYL,1995-11-29 00:00:00,3.0,1995-02-10,71.0,NU,DENY
8,ASYL,1995-04-14 00:00:00,3.0,1995-02-12,61.0,NU,DENY
9,ASYL,1995-07-25 00:00:00,3.0,1995-02-17,70.0,NU,DENY


In [14]:
# I thikn the na is from nat and tracID, since we dealt with all the other columns in our previous cleaning file
df = df.dropna()

In [15]:
df.count()

Appl_Code         597954
Appl_Recd_Date    597954
numAppsPerProc    597954
osc_date          597954
tracid            597954
nat               597954
dec               597954
dtype: int64

In [18]:
# one hot encode categorical variables
# save the last 20% and don't touch it!
# ridge regression
# walk forward validation
from sklearn import preprocessing

categorical = df[['tracid', 'nat', 'dec']]